In [1]:
import pandas as pd

In [2]:
projects = pd.read_pickle('../../data/01_data_from_db/projects.pkl')
projects

,project_id,name,language,created_at
0,1,akka,Scala,2009-02-16 12:51:54
1,2,devtools,R,2010-05-03 04:08:49
2,3,ProjectTemplate,R,2010-08-24 17:22:36
3,4,stat-cookbook,R,2012-04-23 20:24:37
4,5,hiphop-php,C++,2010-01-02 01:17:06
...,...,...,...,...
108713,108736,rails,Ruby,2011-04-29 17:09:06
108714,108737,rails,Ruby,2013-10-03 20:50:33
108715,108738,rails,Ruby,2013-10-01 11:21:03
108716,108739,rails,Ruby,2013-09-27 17:53:25


Połączenie projektów w miesięczne szeregi

In [3]:
months = pd.Series(list(range(1, 13)), name='month')
months

0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
Name: month, dtype: int64

In [4]:
years = pd.Series(list(range(projects.created_at.min().year, projects.created_at.max().year + 1)), name='year')
years

0    2008
1    2009
2    2010
3    2011
4    2012
5    2013
Name: year, dtype: int64

In [5]:
def cartesian_product(left, right):
    return (
       left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1))

In [6]:
projects_time_series = cartesian_product(projects, years.to_frame('year'))

In [7]:
projects_time_series = cartesian_product(projects_time_series, months.to_frame('month'))

In [8]:
projects_time_series

,project_id,name,language,created_at,year,month
0,1,akka,Scala,2009-02-16 12:51:54,2008,1
1,1,akka,Scala,2009-02-16 12:51:54,2008,2
2,1,akka,Scala,2009-02-16 12:51:54,2008,3
3,1,akka,Scala,2009-02-16 12:51:54,2008,4
4,1,akka,Scala,2009-02-16 12:51:54,2008,5
...,...,...,...,...,...,...
7820347,108740,rails,Ruby,2013-09-26 14:38:42,2013,8
7820348,108740,rails,Ruby,2013-09-26 14:38:42,2013,9
7820349,108740,rails,Ruby,2013-09-26 14:38:42,2013,10
7820350,108740,rails,Ruby,2013-09-26 14:38:42,2013,11


Dodanie kolumny informującej, który miesiąc życia projektu badamy

In [9]:
months_from_create = (projects_time_series.year - projects_time_series.created_at.dt.year) * 12 + (projects_time_series.month - projects_time_series.created_at.dt.month)

In [10]:
projects_time_series['months_from_create'] = months_from_create

In [11]:
projects_time_series

,project_id,name,language,created_at,year,month,months_from_create
0,1,akka,Scala,2009-02-16 12:51:54,2008,1,-13
1,1,akka,Scala,2009-02-16 12:51:54,2008,2,-12
2,1,akka,Scala,2009-02-16 12:51:54,2008,3,-11
3,1,akka,Scala,2009-02-16 12:51:54,2008,4,-10
4,1,akka,Scala,2009-02-16 12:51:54,2008,5,-9
...,...,...,...,...,...,...,...
7820347,108740,rails,Ruby,2013-09-26 14:38:42,2013,8,-1
7820348,108740,rails,Ruby,2013-09-26 14:38:42,2013,9,0
7820349,108740,rails,Ruby,2013-09-26 14:38:42,2013,10,1
7820350,108740,rails,Ruby,2013-09-26 14:38:42,2013,11,2


Usunięcie wierszy, w których dodane zostały rok i miesiąc sprzed utworzenia projektu

In [12]:
projects_time_series.drop(projects_time_series[projects_time_series.months_from_create < 0].index, inplace=True)

In [13]:
projects_time_series

,project_id,name,language,created_at,year,month,months_from_create
13,1,akka,Scala,2009-02-16 12:51:54,2009,2,0
14,1,akka,Scala,2009-02-16 12:51:54,2009,3,1
15,1,akka,Scala,2009-02-16 12:51:54,2009,4,2
16,1,akka,Scala,2009-02-16 12:51:54,2009,5,3
17,1,akka,Scala,2009-02-16 12:51:54,2009,6,4
...,...,...,...,...,...,...,...
7820279,108739,rails,Ruby,2013-09-27 17:53:25,2013,12,3
7820348,108740,rails,Ruby,2013-09-26 14:38:42,2013,9,0
7820349,108740,rails,Ruby,2013-09-26 14:38:42,2013,10,1
7820350,108740,rails,Ruby,2013-09-26 14:38:42,2013,11,2


In [14]:
projects_time_series.to_pickle('../../data/02_prepared_data/projects_time_series.pkl')